In [ ]:
# !pip install -q transformers datasets huggingface_hub trl peft accelerate scipy sentencepiece accelerate
# !pip install -i https://pypi.org/simple/ bitsandbytes

### import module

In [1]:
import os
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments,BitsAndBytesConfig
from datasets import load_dataset
from trl import SFTTrainer
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from unsloth import FastLanguageModel

In [2]:
import huggingface_hub
huggingface_hub.notebook_login()

In [5]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: NVIDIA GeForce RTX 4070 SUPER. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaR

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,     # Choose an number > 0, Suggested 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"]
    lora_alpha=16,
    lora_dropout=0, # Supports any, but 0 is optimized
    bias="none",  # Supports any, but =
)

### Dataset 확인

In [3]:
dataset = load_dataset('richard-park/kor_en_panyo_corpus', split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
dataset[0]

{'korean': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다. 이 목록은 명료화를 위해 두 부분으로 나뉘어 있다. 두 목록은 모두 가나다 순이다. 일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다. 이 목록은 주권을 주장하고 점유한 영토를 관리하고 있으나, 적법성의 논란으로 인하여 많은 국가와 외교관계를 맺지 못한 나라를 설명하고 있다. 극소형 국가는 이 목록에 포함하지 않는다. 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정에 따르면, 국가는 다음의 조건을 만족해야 한다. 특히, 마지막 조건은 국제 공동체의 참여 용인을 내포하고 있기 때문에, 다른 나라의 승인이 매우 중요한 역할을 할 수 있다. 이 목록에 포함된 모든 국가는 보통 이 기준을 만족하는 것으로 보이는 자주적이고 독립적인 국가이다. 하지만 몬테비데오 협약 기준을 만족하는지의 여부는 많은 국가가 논쟁이 되고 있는 실정이다. 또한, 몬테비데오 협약 기준만이 국가 지위의 충분한 자격이든 아니든, 국제법의 견해 차이는 존재할 수 있다. 이 물음에 대한 다른 이론에 대한 고리는 아래에서 볼 수 있다. 위 기준에 논거하여 이 목록은 다음 206개 국가를 포함하고 있다. 위 목록에 포함되지 않은 다음 국가는 몬테비데오 협약의 모든 조건을 만족하지 못하거나, 자주적이고 독립적임을 주장하지 않는 국가이다.',
 'pangyo': "This document는 전세계의 206개 countries를 list하고 있습니다. 이는 countries의 현황과 sovereignty approval 정보를 보여 줍니다. list는 논란이 될 수 있는 국가 자격에 대해 논의하고 있

### json format을 instruction dataset format으로 변환 (alpaca format)

In [ ]:
dataset['korean'][1]

'체첸 공화국, 또는 줄여서 체첸은 러시아의 공화국이다. 체첸에서 사용되는 언어는 체첸어와 러시아어이다. 체첸어는 캅카스제어 중, 북동 캅카스제어로 불리는 그룹에 속하는데 인구시어와 매우 밀접한 관계에 있다. 거의 대부분이 체첸인이다. 일부는 러시아인, 인구시인과 기타 북코카서스계 민족도 섞여있다. 1989년에 행해진 체첸-인구시 자치공화국의 통계에서는 체첸인이 956,879명, 인구시인이 237,438명으로, 269,000명의 러시아인은 인구의 약 23%로 소수 민족이었다. 그 후 서부가 잉구슈 공화국으로 분리되었기 때문에 인구시인들의 수가 절반 가까이 감소하고, 내전과 사회불안으로 대부분의 러시아인은 체첸공화국에서 대부분 떠났다. 1990년대 체첸 공화국에 남아 있던 러시아인은 약 6만 명이었다. 체첸 공화국은 일반적으로 러시아 연방 중에서도 젊은 층이 가장 많은 인구 구성을 가진다. 1990년대에는 몇몇 지방에서 인구증가가 있었다. 다게스탄 지역을 통해 16세기에서 19세기를 기점으로, 체첸인들은 대다수가 수니파 이슬람교를 믿으며 러시아 정교회도 소수 존재한다. 1830년에서 1859년에 이르는 동안, 러시아 군은 오스만 투르크 제국과의 접경지역 안보를 이유로 체첸에 진주했다. 체첸은 1859년 러시아 제국에 병합됐다. 제2차 세계 대전 동안 소비에트 정부는 체첸이 나치군과 협력했다고 맹비난하였다. 스탈린은 체첸 국민 전체에게 카자흐스탄으로의 강제이주를 명령했다. 그 후 스탈린이 사망한 지 4년 후인 1957년에 이르러서야 체첸인의 귀환이 허용되었다. 그리고 체첸전쟁때 러시아의 엄청난 공세로 인해 체첸 전역은 초토화 되었다.'

In [ ]:
def format_instruction(dataset):
  output_texts = []
  for i in range(len(dataset)):
    text = f"""### Instruction:
      Convert the Korean sentences below into Korean-English code-switching sentences.

      ### Input:
      {dataset['korean'][i]}

      ### Response:
      {dataset['pangyo'][i]}
      """
    output_texts.append(text)
  return output_texts


In [ ]:
ds = dataset[:2]
ll = format_instruction(ds)
ll

['### Instruction:\n      Convert the Korean sentences below into Korean-English code-switching sentences.\n\n      ### Input:\n      이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다. 이 목록은 명료화를 위해 두 부분으로 나뉘어 있다. 두 목록은 모두 가나다 순이다. 일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다. 이 목록은 주권을 주장하고 점유한 영토를 관리하고 있으나, 적법성의 논란으로 인하여 많은 국가와 외교관계를 맺지 못한 나라를 설명하고 있다. 극소형 국가는 이 목록에 포함하지 않는다. 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정에 따르면, 국가는 다음의 조건을 만족해야 한다. 특히, 마지막 조건은 국제 공동체의 참여 용인을 내포하고 있기 때문에, 다른 나라의 승인이 매우 중요한 역할을 할 수 있다. 이 목록에 포함된 모든 국가는 보통 이 기준을 만족하는 것으로 보이는 자주적이고 독립적인 국가이다. 하지만 몬테비데오 협약 기준을 만족하는지의 여부는 많은 국가가 논쟁이 되고 있는 실정이다. 또한, 몬테비데오 협약 기준만이 국가 지위의 충분한 자격이든 아니든, 국제법의 견해 차이는 존재할 수 있다. 이 물음에 대한 다른 이론에 대한 고리는 아래에서 볼 수 있다. 위 기준에 논거하여 이 목록은 다음 206개 국가를 포함하고 있다. 위 목록에 포함되지 않은 다음 국가는 몬테비데오 협약의 모든 조건을 만족하지 못하거나, 자주적이고 독립적임을 주장하지 않는 국가이다.\n\n      ### Response:\n 

### Load the model in 4-bit quantization

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
model_name="meta-llama/Llama-2-7b-chat-hf"
base_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, quantization_config=bnb_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
base_model.config.use_cache = False
base_model = prepare_model_for_kbit_training(base_model)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weried overflow issue with fp16 training

### Change the LORA hyperparameters accordingly to fit your use case

In [ ]:
import torch
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    return list(lora_module_names)


def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [ ]:
peft_config = LoraConfig(
    r=128,
    lora_alpha=16,
    target_modules=find_all_linear_names(base_model),
    lora_dropout=0.05,
    bias="none",
    task_type="CASUAL_LM"
)

In [ ]:
base_model = get_peft_model(base_model, peft_config)
print_trainable_parameters(base_model)

trainable params: 319815680 || all params: 3820228608 || trainables%: 8.371637219046761


### Parameters for training arguments details

In [ ]:
output_dir="./results"
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    max_grad_norm=0.3,
    num_train_epochs=100,
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    save_total_limit=3,
    logging_steps=10,
    output_dir=output_dir,
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05
)

In [ ]:
trainer = SFTTrainer(
    base_model,
    train_dataset=dataset,
    tokenizer=tokenizer,
    max_seq_length=2048,
    formatting_func=format_instruction,
    args=training_args
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.365700
20,1.078000
30,0.819200


KeyboardInterrupt: 